/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 1 of the case study, which means that there'll be more guidance for you about how to 
setup your local SQLite connection in PART 2 of the case study. 

The questions in the case study are exactly the same as with Tier 2. 

# PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */


# QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT name 
FROM Facilities
WHERE membercost > 0;


/* Q2: How many facilities do not charge a fee to members? */

4 Facilities


/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT facid, name, membercost, monthlymaintenance 
FROM Facilities 
WHERE membercost > 0 
    AND membercost < (monthlymaintenance * 0.20);


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT * 
FROM Facilities 
WHERE facid IN (1, 5);

or 

SELECT * 
FROM Facilities 
WHERE name LIKE '%2';


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name, monthlymaintenance, 
    CASE WHEN monthlymaintenance > 100 THEN 'expensive' 
    ELSE 'cheap' END 
    AS label 
FROM Facilities;


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT firstname, surname
FROM Members 
WHERE joindate = '2012-09-26 18:08:45';


/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT DISTINCT CONCAT(m.firstname, ' ' , m.surname) AS member, f.name AS court
FROM Members AS m
INNER JOIN Bookings AS b
	ON m.memid = b.memid
INNER JOIN Facilities AS f
	ON b.facid = f.facid
WHERE f.facid IN (0,1) 
	AND m.memid > 0
ORDER BY `member` ASC;

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT CONCAT(m.firstname, ' ' , m.surname) AS member, f.name AS facility, 
CASE
	WHEN m.memid = 0 THEN (f.guestcost * b.slots)
    ELSE (f.membercost * b.slots)
    END AS cost
FROM Members AS m
INNER JOIN Bookings AS b
	ON m.memid = b.memid
INNER JOIN Facilities AS f
	ON b.facid = f.facid
WHERE 
	b.starttime >= '2012-09-14' AND 
    b.starttime < '2012-09-15' AND (
        (m.memid = 0 AND f.guestcost * b.slots > 30) OR 
        (m.memid != 0 AND f.membercost * b.slots > 30)
                                                       )
ORDER BY cost DESC;


/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT member, facility, cost
FROM (
        SELECT CONCAT(m.firstname, ' ' , m.surname) AS member, f.name AS facility, 
        CASE
            WHEN m.memid = 0 THEN (f.guestcost * b.slots)
            ELSE (f.membercost * b.slots)
            END AS cost
        FROM Members AS m
        INNER JOIN Bookings AS b
            ON m.memid = b.memid
        INNER JOIN Facilities AS f
            ON b.facid = f.facid
        WHERE 
            b.starttime >= '2012-09-14' AND 
            b.starttime < '2012-09-15' 
            ) AS b
            WHERE cost > 30
ORDER BY cost DESC;


/* PART 2: SQLite
/* We now want you to jump over to a local instance of the database on your machine. 

Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 

Make sure that the SQLFiles folder containing thes files is in your working directory, and
that you haven't changed the name of the .db file from 'sqlite\db\pythonsqlite'.

You should see the output from the initial query 'SELECT * FROM FACILITIES'.

Complete the remaining tasks in the Jupyter interface. If you struggle, feel free to go back
to the PHPMyAdmin interface as and when you need to. 

You'll need to paste your query into value of the 'query1' variable and run the code block again to get an output.

In [8]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()


2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [ ]:
SELECT * 
        FROM (
                SELECT subq.facility, SUM( subq.cost ) AS totalrevenue
                FROM (
                    SELECT f.name AS facility, 
                        CASE WHEN b.memid =0
                        THEN f.guestcost * b.slots
                        ELSE f.membercost * b.slots
                        END AS cost
                FROM Bookings AS b
                INNER JOIN Facilities AS f
                    ON b.facid = f.facid
                INNER JOIN Members AS m
                    ON b.memid = m.memid
                ) AS subq
        GROUP BY subq.facility
        ) AS subq2
        WHERE subq2.totalrevenue <1000;

In [15]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT * 
        FROM (
                SELECT subq.facility, SUM( subq.cost ) AS totalrevenue
                FROM (
                    SELECT f.name AS facility, 
                        CASE WHEN b.memid =0
                        THEN f.guestcost * b.slots
                        ELSE f.membercost * b.slots
                        END AS cost
                FROM Bookings AS b
                INNER JOIN Facilities AS f
                    ON b.facid = f.facid
                INNER JOIN Members AS m
                    ON b.memid = m.memid
                ) AS subq
        GROUP BY subq.facility
        ) AS subq2
        WHERE subq2.totalrevenue <1000;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()


2.6.0
2. Query all tasks
('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [ ]:
SELECT m1.surname, m1.firstname, m2.recommendedby AS recommender_ID, m2.surname AS recommender_surname, m2.firstname AS recommender_firstname
FROM Members AS m1
	LEFT JOIN Members AS m2
    	ON m1.memid = m2.recommendedby
     WHERE m1.recommendedby != 0
ORDER BY m1.surname, m1.firstname;

In [17]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT m1.surname, m1.firstname, m2.recommendedby AS recommender_ID, m2.surname AS recommender_surname, m2.firstname AS recommender_firstname
FROM Members AS m1
	LEFT JOIN Members AS m2
    	ON m1.memid = m2.recommendedby
     WHERE m1.recommendedby != 0
ORDER BY m1.surname, m1.firstname;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()


2.6.0
2. Query all tasks
('Bader', 'Florence', '15', 'Sarwin', 'Ramnaresh')
('Baker', 'Anne', None, None, None)
('Baker', 'Timothy', '16', 'Coplin', 'Joan')
('Boothe', 'Tim', None, None, None)
('Butters', 'Gerald', '5', 'Genting', 'Matthew')
('Coplin', 'Joan', None, None, None)
('Crumpet', 'Erica', None, None, None)
('Dare', 'Nancy', None, None, None)
('Farrell', 'David', None, None, None)
('Farrell', 'Jemima', '13', 'Baker', 'Timothy')
('Farrell', 'Jemima', '13', 'Pinker', 'David')
('GUEST', 'GUEST', None, None, None)
('Genting', 'Matthew', '20', 'Rumney', 'Henrietta')
('Hunt', 'John', None, None, None)
('Jones', 'David', '11', 'Jones', 'Douglas')
('Jones', 'Douglas', None, None, None)
('Joplette', 'Janice', '4', 'Dare', 'Nancy')
('Joplette', 'Janice', '4', 'Jones', 'David')
('Mackenzie', 'Anna', None, None, None)
('Owen', 'Charles', None, None, None)
('Pinker', 'David', None, None, None)
('Purview', 'Millicent', '30', 'Hunt', 'John')
('Rownam', 'Tim', '3', 'Boothe', 'Tim')
('Rumney',

/* Q12: Find the facilities with their usage by member, but not guests */

In [ ]:
SELECT f.facid, f.name, COUNT(b.facid) AS member_usage
FROM Facilities AS f
	INNER JOIN Bookings AS b
    	ON f.facid = b.facid
    WHERE b.memid != 0
 GROUP BY f.facid, f.name;

In [18]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT f.facid, f.name, COUNT(b.facid) AS member_usage
FROM Facilities AS f
	INNER JOIN Bookings AS b
    	ON f.facid = b.facid
    WHERE b.memid != 0
 GROUP BY f.facid, f.name;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()


2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 308)
(1, 'Tennis Court 2', 276)
(2, 'Badminton Court', 344)
(3, 'Table Tennis', 385)
(4, 'Massage Room 1', 421)
(5, 'Massage Room 2', 27)
(6, 'Squash Court', 195)
(7, 'Snooker Table', 421)
(8, 'Pool Table', 783)


/* Q13: Find the facilities usage by month, but not guests */

In [ ]:
SELECT strftime('%m', starttime) AS month, f.name, COUNT(b.facid) AS member_usage
FROM Facilities AS f
	INNER JOIN Bookings AS b
    	ON f.facid = b.facid
    WHERE b.memid != 0
 GROUP BY month, f.name;

In [30]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT strftime('%m', starttime) AS month, f.name, COUNT(b.facid) AS member_usage
        FROM Facilities AS f
            INNER JOIN Bookings AS b
                ON f.facid = b.facid
            WHERE b.memid != 0
         GROUP BY month, f.name;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()


2.6.0
2. Query all tasks
('07', 'Badminton Court', 51)
('07', 'Massage Room 1', 77)
('07', 'Massage Room 2', 4)
('07', 'Pool Table', 103)
('07', 'Snooker Table', 68)
('07', 'Squash Court', 23)
('07', 'Table Tennis', 48)
('07', 'Tennis Court 1', 65)
('07', 'Tennis Court 2', 41)
('08', 'Badminton Court', 132)
('08', 'Massage Room 1', 153)
('08', 'Massage Room 2', 9)
('08', 'Pool Table', 272)
('08', 'Snooker Table', 154)
('08', 'Squash Court', 85)
('08', 'Table Tennis', 143)
('08', 'Tennis Court 1', 111)
('08', 'Tennis Court 2', 109)
('09', 'Badminton Court', 161)
('09', 'Massage Room 1', 191)
('09', 'Massage Room 2', 14)
('09', 'Pool Table', 408)
('09', 'Snooker Table', 199)
('09', 'Squash Court', 87)
('09', 'Table Tennis', 194)
('09', 'Tennis Court 1', 132)
('09', 'Tennis Court 2', 126)
